# Descrição dos Modelos utlizados nas análises

Na fase de análise dos dados de medição foram empregados 3 modelos distintos, sendo eles: LSTM (Long-Short Term Memory)[https://keras.io/getting-started/sequential-model-guide/], KNN(K-nearest Neighbors)[http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor] e SARIMAX(Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors model)[http://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html]. Os dados utilizados até então totalizam em torno de 4 meses de medições, sendo a analise atual obtida a partir de médias de 30min. Nós dividimos as análise em duas abordagens centrais. A primeira consiste de um forecasting de tamanho n > 1 (e.g. 24) no qual o resultado da predição p é entrada para o cálculo da predição p+1, p < n. Na segunda abordagem denominada janela deslizante, para efetuar a análise da performance do nosso modelo, vamos efetuando a predição de um valor a frente, calculamos o erro dado que temos o valor target e vamos "andando" com um "janela" sobre o conjunto de teste. O erro em ambas abordagens é o erro quadrado, que é acumulado e apresentado no final da análise. Na abordagem por realimentação o dataset é da forma [X,y], onde X[i] é da forma [24 valores anteriores, próximo valor]. Já na abordagem por janelas deslizantes, o dataset contém em X, valores de 48 pontos anteriores e y o próximo valor, ou seja, X[i] é da forma  [48 valores anteriores, próximo valor].

# Parametros dos modelos
Para ambas as abordagens 80% dos dataset é utilizado como treino e 20% para teste. No entanto, na abordagem por janelas deslizantes os k valores finais do conjunto de treino são usados no conjunto de teste. Os parâmetros foram estudados previamente para a escolha dos melhores valores para cada modelo, porém um estudo mais aprofundado sobre os parâmetros específicos do modelo ARIMA e como obtê-los de maneira automatizada é válido.
### Metodologia retroaliamentada
**KNN**  : 
    1. n_neighbors = 1, 
    2. weights='uniform', 
    3. algorithm='auto', 
    4. leaf_size=30, 
    5. p=2, 
    6. metric='minkowski'
    
**LSTM** :
    1. Sequential()
    2. LSTM(units=10, input_dim=24)
    3. Dense(units=1) # essa camada efetua output = activation(dot(input, kernel) + bias) 
    4. optimizer='adam'
    5. loss='mean_squared_error'
    6. nb_epoch=49
    7. batch_size=128
    
**SARIMAX**:
    1. order=(0, 1, 1)
    2. seasonal_order=(0,1,1,48), 
    3. trend='n',
    4. mle_regression=True, 
    5. enforce_stationarity=True, 
    6. enforce_invertibility=True

### Metodologia janelas deslizantes
**KNN**  : 
    1. n_neighbors = 1, 
    2. weights='uniform', 
    3. algorithm='auto',
    4. leaf_size=30,
    5. p=2, 
    6. metric='minkowski'
    
**LSTM** :
    1. Sequential()
    2. LSTM(units=10, input_dim=48)
    3. Dense(units=1) 
    4. optimizer='adam'
    5. loss='mean_squared_error'
    6. nb_epoch=49
    7. batch_size=128
    
**SARIMAX**:
    1. order=(0, 1, 1)
    2. seasonal_order=(0,1,1,48), 
    3. trend='n',
    4. mle_regression=True, 
    5. enforce_stationarity=True, 
    6. enforce_invertibility=True



Na abordagem por janelas deslizantes, o erro é calculado com relação a **todo o conjunto de teste**, enquanto que na abordagem por realimentação efetua-se o forecasting de tamanho 48 (24h) e calcula-se o erro com relação **aos primeiros 48 pontos do conjunto de teste**.
**NOTE**: LSTMs são sensiveis a escala do dado de entrada, especificamente quando a função de ativação utilizada é a sigmoid(default). Portanto, seguindo uma boa prática, efetuamos uma transformação de escala para valores entre (0,1), usando a função sklearn.preprocessing.MinMaxScaler. Após o treinameto da rede os valores são reajustados a escala inicial.